In [1]:
import sys
import csv
import random
from tqdm import tqdm
import cv2
import datetime
import h5py
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Conv2D, Cropping2D, Dropout, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split
#TensorFlowがGPUを認識しているか確認
import tensorflow as tf
tf.test.gpu_device_name()

Using TensorFlow backend.
/root/miniconda3/envs/zcar01/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


'/gpu:0'

In [2]:
data_dir = "data/"
raw_driving_log_name = "driving_log.csv"

raw_driving_log_path = data_dir + raw_driving_log_name
driving_log_name = "driving_log2.csv"
driving_file_path = data_dir + driving_log_name

In [3]:
# driving_logファイルのPATHを修正
def convert2filepath(local_path):
    start_index = local_path.find("IMG")
    converted_path = data_dir + local_path[start_index::]
    return converted_path

df = pd.read_csv(raw_driving_log_path, header=None, names=('center_path', 'left_path', 'right_path', "degree", "throttle", "brake", "speed"))
df["center_path"] = df["center_path"].map(convert2filepath)
df["left_path"] = df["left_path"].map(convert2filepath)
df["right_path"] = df["right_path"].map(convert2filepath)
df.to_csv(driving_file_path, header=False, index=False)

In [4]:
def getrowsFromDrivingLogs(dataPath):
    rows = []
    with open(dataPath + "/" + driving_log_name) as csvFile:
        reader = csv.reader(csvFile)            
        for row in reader:
            rows.append(row)
    return rows

def getImageArray3angle(imagePath, steering, images, steerings):
    originalImage = cv2.imread(imagePath.strip())
    image = cv2.cvtColor(originalImage, cv2.COLOR_BGR2RGB)
    images.append(image)
    steerings.append(steering)

def getReverseImageArray3angle(imagePath, steering, images, steerings):
    originalImage = cv2.imread(imagePath.strip())
    image = cv2.cvtColor(originalImage, cv2.COLOR_BGR2RGB)
    reversed_image = cv2.flip(image,1)
    steering_flipped = - steering # ステアリング角度が左折状態になっているので、左右逆転させることで右折にします。

    images.append(reversed_image)
    steerings.append(steering_flipped) 
    
def getImagesAndSteerings(rows):
    
    images = []
    steerings = []
    
    for row in rows:
        #角度
        steering = float(row[3])
        # 左右のカメラのステアリング測定値を調整します
        parameter = 0.2 
        # このパラメータが調整用の値です。
        # 左のカメラはステアリング角度が実際よりも低めに記録されているので、少し値を足してやります。右のカメラはその逆です。
        steering_left = steering + parameter
        steering_right = steering - parameter

        getImageArray3angle(row[0], steering, images, steerings)
        getImageArray3angle(row[1], steering_left, images, steerings)
        getImageArray3angle(row[2], steering_right, images, steerings)
        getReverseImageArray3angle(row[0], steering, images, steerings)
        getReverseImageArray3angle(row[1], steering_left, images, steerings)
        getReverseImageArray3angle(row[2], steering_right, images, steerings)

    return (np.array(images), np.array(steerings))

def trainModelAndSave(model, train_data, validation_data, epochs, batch_size):
    # 学習パラメータ
    loss = 'mean_squared_error'
    learn_rate = 1.0e-4
    optimizer = RMSprop(learn_rate)
    now = datetime.datetime.now()
    filename = now.strftime('%Y%m%d_%H%M%S') 

    # compile and train the model using the generator function
    train_size = len(train_data)*2*3
    valid_size = len(validation_data)*2*3

    file_path_model = filename + "_model-weights.hdf5"
    checkpoint = ModelCheckpoint(file_path_model, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
    early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')  
    callbacks_list = [checkpoint, early_stopping]

    #モデルの設定
    model.compile(loss=loss, optimizer=optimizer)
    #モデルの学習
    history_object = model.fit_generator(batch_generator(train_data, batch_size=batch_size),
                                         samples_per_epoch=train_size,
                                         validation_data=batch_generator(validation_data, batch_size=batch_size), 
                                         nb_val_samples=valid_size, nb_epoch=epochs, verbose=1,
                                         callbacks = callbacks_list)
    #モデルの保存
    model.save(filename + '_model.h5')
    return history_object

def loadCSVData():
    data = []
    with open(driving_file_path) as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            data.append(line)
            
    train_data, validation_data = train_test_split(data, test_size=0.2, random_state=0)
    return train_data, validation_data

def batch_generator(input_data, batch_size):
    # ジェネレータが停止しないようにループさせる
    while True: 
        for offset in range(0, len(input_data), batch_size):
            batch_data = input_data[offset:offset+batch_size]
            images = []
            steerings = []

            images, steerings = getImagesAndSteerings(batch_data)
            yield np.array(images), np.array(steerings)

#NVIDIA
def nvidia_model():
    model = Sequential()
    model.add(Cropping2D(cropping=((50,20), (1,1)), input_shape=(160,320,3)))
    model.add(Lambda(lambda x: (x / 255.0) - 0.5))
    model.add(Conv2D(24,5,5, subsample=(2,2), activation='elu'))
    model.add(Conv2D(36,5,5, subsample=(2,2), activation='elu'))
    model.add(Conv2D(48,5,5, subsample=(2,2), activation='elu'))
    model.add(Conv2D(64,3,3, activation='elu'))
    model.add(Conv2D(64,3,3, activation='elu'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    model.add(Dense(1))
    return model

In [5]:
epochs = 10
#バッチサイズは40以下にしてください。
batch_size = 40

now = datetime.datetime.now()
filename = now.strftime('%Y%m%d_%H%M%S') 

train_data, validation_data = loadCSVData()
sampled_train_data = random.sample(train_data, int(len(train_data)))
sampled_validation_data = random.sample(validation_data, int(len(validation_data)))

print('Training data:', len(train_data))
print('Training label:', len(validation_data))

#モデルの指定
model = nvidia_model()
#モデルの訓練と保存
history_object = trainModelAndSave(model, sampled_train_data, 
                                   sampled_validation_data, epochs, batch_size)

Training data: 2165
Training label: 542
Epoch 1/10
12990/12990 [==============================] - 246s - loss: 0.0770 - val_loss: 0.0771
Epoch 2/10
12990/12990 [==============================] - 148s - loss: 0.0715 - val_loss: 0.0727
Epoch 3/10
12990/12990 [==============================] - 148s - loss: 0.0688 - val_loss: 0.0709
Epoch 4/10
12990/12990 [==============================] - 145s - loss: 0.0669 - val_loss: 0.0705
Epoch 5/10
12990/12990 [==============================] - 147s - loss: 0.0657 - val_loss: 0.0702
Epoch 6/10
12990/12990 [==============================] - 147s - loss: 0.0642 - val_loss: 0.0703
Epoch 7/10
12990/12990 [==============================] - 143s - loss: 0.0628 - val_loss: 0.0708
Epoch 8/10
12990/12990 [==============================] - 147s - loss: 0.0611 - val_loss: 0.0669
Epoch 9/10
12990/12990 [==============================] - 148s - loss: 0.0595 - val_loss: 0.0659
Epoch 10/10
12990/12990 [==============================] - 147s - loss: 0.0578 - val_lo